In [70]:
using LinearAlgebra
using AbstractTrees
using DataFrames
using CSV
include("structs.jl")
include("trees.jl")
include("sourceQt.jl")
include("sourceCl.jl")

p_rand_yev (generic function with 2 methods)

In [41]:
ch = channel(kron(gates["Hadamard"], Matrix(I, 2,2)), "Z")

Circuit = [pauli_proj_meas["Mx"]["A"], pauli_proj_meas["Mx"]["B"], ch, pauli_proj_meas["Mx"]["A"], pauli_proj_meas["Mx"]["B"]];

In [71]:
root = create_tree(Circuit, density_matrix([1 0 0 0; 0 1 0 0; 0 0 1 0;0 0 0 1], "Z"))
b = create_tree(Circuit, density_matrix([1 0 0 0; 0 1 0 0; 0 0 1 0;0 0 0 1], "Z"))

TreeNode(1, nothing, density_prob(1.0, ComplexF64[1.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 1.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im 1.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im 1.0 + 0.0im], 0), TreeNode(2, TreeNode(#= circular reference @-2 =#), density_prob(1.9999999999999991, ComplexF64[0.25 + 0.0im 0.0 + 0.0im 0.25 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.25 + 0.0im 0.0 + 0.0im 0.25 + 0.0im; 0.25 + 0.0im 0.0 + 0.0im 0.25 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.25 + 0.0im 0.0 + 0.0im 0.25 + 0.0im], 1), TreeNode(4, TreeNode(#= circular reference @-2 =#), density_prob(0.4999999999999998, ComplexF64[0.25 + 0.0im 0.25 + 0.0im 0.25 + 0.0im 0.25 + 0.0im; 0.25 + 0.0im 0.25 + 0.0im 0.25 + 0.0im 0.25 + 0.0im; 0.25 + 0.0im 0.25 + 0.0im 0.25 + 0.0im 0.25 + 0.0im; 0.25 + 0.0im 0.25 + 0.0im 0.25 + 0.0im 0.25 + 0.0im], 1), TreeNode(8, TreeNode(#= circular reference @-2 =#), density_prob(0.0, ComplexF64[0.4999999999999999 + 0.0im 0.4999999999999999 + 0.

In [57]:
b.left.left.right.left.value.prob

0.49999999999999967

In [72]:
trimming!(root)

1.0
1.9999999999999991


0.4999999999999998
1.0
0.49999999999999967
0.9999999999999996
0.49999999999999967
0.9999999999999996
0.4999999999999998
1.0
0.49999999999999967
0.9999999999999996
0.49999999999999967
0.9999999999999996
1.9999999999999991
0.4999999999999998
1.0
0.49999999999999967
0.9999999999999996
0.49999999999999967
0.9999999999999996
0.4999999999999998
1.0
0.49999999999999967
0.9999999999999996
0.49999999999999967
0.9999999999999996


27-element Vector{Any}:
 TreeNode(1, nothing, density_prob(1.0, ComplexF64[1.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 1.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im 1.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im 1.0 + 0.0im], 0), TreeNode(2, TreeNode(#= circular reference @-2 =#), density_prob(1.9999999999999991, ComplexF64[0.25 + 0.0im 0.0 + 0.0im 0.25 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.25 + 0.0im 0.0 + 0.0im 0.25 + 0.0im; 0.25 + 0.0im 0.0 + 0.0im 0.25 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.25 + 0.0im 0.0 + 0.0im 0.25 + 0.0im], 1), TreeNode(4, TreeNode(#= circular reference @-2 =#), density_prob(0.4999999999999998, ComplexF64[0.25 + 0.0im 0.25 + 0.0im 0.25 + 0.0im 0.25 + 0.0im; 0.25 + 0.0im 0.25 + 0.0im 0.25 + 0.0im 0.25 + 0.0im; 0.25 + 0.0im 0.25 + 0.0im 0.25 + 0.0im 0.25 + 0.0im; 0.25 + 0.0im 0.25 + 0.0im 0.25 + 0.0im 0.25 + 0.0im], 1), nothing, TreeNode(9, TreeNode(#= circular reference @-2 =#), density_prob(1.0, ComplexF64[0.49999999999999

TreeNode(30, TreeNode(15, TreeNode(7, TreeNode(3, TreeNode(1, nothing, density_prob(1.0, ComplexF64[1.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 1.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im 1.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im 1.0 + 0.0im], 0), TreeNode(2, TreeNode(#= circular reference @-2 =#), density_prob(1.9999999999999991, ComplexF64[0.25 + 0.0im 0.0 + 0.0im 0.25 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.25 + 0.0im 0.0 + 0.0im 0.25 + 0.0im; 0.25 + 0.0im 0.0 + 0.0im 0.25 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.25 + 0.0im 0.0 + 0.0im 0.25 + 0.0im], 1), TreeNode(4, TreeNode(#= circular reference @-2 =#), density_prob(0.4999999999999998, ComplexF64[0.25 + 0.0im 0.25 + 0.0im 0.25 + 0.0im 0.25 + 0.0im; 0.25 + 0.0im 0.25 + 0.0im 0.25 + 0.0im 0.25 + 0.0im; 0.25 + 0.0im 0.25 + 0.0im 0.25 + 0.0im 0.25 + 0.0im; 0.25 + 0.0im 0.25 + 0.0im 0.25 + 0.0im 0.25 + 0.0im], 1), nothing, TreeNode(9, TreeNode(#= circular reference @-2 =#), density_prob(1.0, Co

In [35]:
root.left.right.left

TreeNode(10, TreeNode(5, TreeNode(2, TreeNode(1, nothing, density_prob(1.0, ComplexF64[1.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 1.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im 1.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im 1.0 + 0.0im], 0), TreeNode(#= circular reference @-2 =#), TreeNode(3, TreeNode(#= circular reference @-2 =#), density_prob(1.9999999999999991, ComplexF64[0.25 + 0.0im 0.0 + 0.0im -0.25 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.25 + 0.0im 0.0 + 0.0im -0.25 + 0.0im; -0.25 + 0.0im 0.0 + 0.0im 0.25 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im -0.25 + 0.0im 0.0 + 0.0im 0.25 + 0.0im], 1), TreeNode(6, TreeNode(#= circular reference @-2 =#), density_prob(0.4999999999999998, ComplexF64[0.25 + 0.0im 0.25 + 0.0im -0.25 + 0.0im -0.25 + 0.0im; 0.25 + 0.0im 0.25 + 0.0im -0.25 + 0.0im -0.25 + 0.0im; -0.25 + 0.0im -0.25 + 0.0im 0.25 + 0.0im 0.25 + 0.0im; -0.25 + 0.0im -0.25 + 0.0im 0.25 + 0.0im 0.25 + 0.0im], 1), TreeNode(12, TreeNode(#= circular referenc

In [101]:
function doall(ms, chann, qt)
    Vectors = [[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1],
            [1,1,0,0],[1,0,1,0],[1,0,0,1],[0,1,1,0],[0,1,0,1],[0,0,1,1],
            [1,1,1,0],[1,1,0,1],[1,0,1,1],[0,1,1,1],[1,1,1,1]]
    States = [state_vector(i./sqrt(sum(i)), "Z") for i in Vectors];
    Measures = [pauli_proj_meas[i][j] for (i,j) in zip(ms, qt)]
    ch = channel(kron(gates[chann], Matrix(I, 2, 2)), "Z")
    n = 100000
    file = "QtTEdata/$(chann)/ch-$(chann)_measurement-$(ms[1])$(qt[1])_$(ms[2])$(qt[2])_$(ms[3])$(qt[3])_$(ms[4])$(qt[4]).csv"
    df = DataFrame()
    df.TE = Float64[]
    df.Initial_State = [];

    for state in States
        Qt = QT_Dynamical_model([[-1,1]],[[-1,1]],state, ch, Measures)
        Source, Target = simulation(Qt, n)
        T = Time_Series(n, [Qt.x;Qt.x], [Qt.y;Qt.y], Target, Source, Qt)
        push!(df.TE, TE(2, 1, 1 ,T, Freq)[1].value)
        push!(df.Initial_State, state.vec)
    end
    
    CSV.write(file, df);
end

doall (generic function with 2 methods)

In [103]:
Measurements = collect(Iterators.product(["Mx", "My", "Mz"],["Mx", "My", "Mz"],["Mx", "My", "Mz"],["Mx", "My", "Mz"]))
chann = "PauliX"
qt = ["A", "B", "A", "B"]
#qt = ["B", "A", "B", "A"]
for ms in Measurements
    doall(ms, chann, qt)
end

In [90]:
chann = "Swap"
ms = Measurements[30]
qt = ["A", "B", "A", "B"]
file = "QtTEdata/$(chann)/ch-$(chann)_1_measurement-$(ms[1])$(qt[1])_$(ms[2])$(qt[2])_$(ms[3])$(qt[3])_$(ms[4])$(qt[4]).csv"
df = CSV.File(file) |> DataFrame;
sum(df.TE)

0.0

In [107]:
A = []
chann = "Hadamard"
for ms in Measurements
    t = ["A", "B", "A", "B"]
    file = "QtTEdata/$(chann)/ch-$(chann)_measurement-$(ms[1])$(qt[1])_$(ms[2])$(qt[2])_$(ms[3])$(qt[3])_$(ms[4])$(qt[4]).csv"
    df = CSV.File(file) |> DataFrame;
    push!(A, sum(df.TE))
end